In [127]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plot
from fastai.tabular.all import *

In [ ]:
dt = "20250103"
nrows = 1_500_000
df = pd.read_csv(f"market/xnas-itch-{dt}.mbp-10.csv", nrows=nrows)
#trddf = pd.read_csv(f"market/xnas-itch-{dt}.trades.csv", nrows=nrows)

df.head()

In [ ]:
df.shape

#### Adjust timestamps + drop outside market hours

In [ ]:
df["ts_event"] = pd.to_datetime(df.ts_event, unit="ns")
df["ts_event"] = df["ts_event"].dt.tz_convert('US/Eastern')
df = df[df["ts_event"].dt.time >= pd.to_datetime("09:30").time()]
df = df[df["ts_event"].dt.time <= pd.to_datetime("16:00").time()]
#df.set_index('ts_event', inplace=True)

df = add_datepart(df, 'ts_event', drop=False)

' '.join(o for o in df.columns if o.startswith('ts_event'))

#### Cleanup (drop rows with NA)

In [ ]:
orig_len = df.shape[0]
df.dropna(inplace=True)
cur_len = df.shape[0]
drop_cnt = orig_len-cur_len
if  drop_cnt > 1000:
    raise Exception(f"{drop_cnt} rows dropped, seems a lot")
else:
    print(f"{drop_cnt} dropped out of {orig_len}")

In [ ]:
# Filter the dataframe for action "T"
df_trades = df[df['action'] == "T"]

# Group by buckets of 15 minutes
df_trades.set_index('ts_event').resample('15min').size().plot(kind='bar', title='Count of Trades Over Time')
plot.xlabel('Time')
#plot.xticks(rotation=45)
labels = [item.get_text().split(' ')[1].split('+')[0][:-3] for item in plot.gca().get_xticklabels()]
plot.gca().set_xticklabels(labels)
plot.ylabel('Count')
plot.show()

In [ ]:
trades = df[(df.action == "T") & (df.side != "N")]
cols = ["action", "side", "price", "size", "bid_sz_00", "bid_px_00", "ask_px_00", "ask_sz_00",
         "bid_sz_01", "bid_px_01", "ask_px_01", "ask_sz_01","sequence"]
trades.side.value_counts()


In [134]:
def display_around(idx, nbrows=6):
    iloc_index = df.index.get_loc(idx)
    print(iloc_index)
    return df.iloc[iloc_index-1:iloc_index+nbrows-1][cols]
idx = 0

In [ ]:
x = display_around(trades.index[idx])
print(idx, trades.index[idx])
idx = idx + 1
x

#### Add spread

In [ ]:
df['spread'] = (df['ask_px_00'] - df['bid_px_00'])/(df['bid_px_00']+df['ask_px_00'])
df['spread'].plot(title='Spread Over Time')

### calculate imbalance

In [137]:
def calc_bid_weight(row):
    return  np.array([row['mid']/(row['mid']-row[f'bid_px_0{i}'])*row[f'bid_sz_0{i}'] for i in range(0,10)]).sum(axis=0)
def calc_ask_weight(row): 
    return np.array([row['mid']/(row[f'ask_px_0{i}']-row['mid'])*row[f'ask_sz_0{i}'] for i in range(0,10)]).sum(axis=0)    

In [ ]:
# want to calculate the amount but also how spread out it is
df['mid'] = (df['bid_px_00']+df['ask_px_00'])/2
df['bid_weight'] = calc_bid_weight(df)
df['ask_weight'] = calc_ask_weight(df)
df['bid_weight_log'] = np.log(df['bid_weight'])
df['ask_weight_log'] = np.log(df['ask_weight'])

df[['bid_weight', 'ask_weight', 'bid_weight_log', 'ask_weight_log']].describe()

In [ ]:

ax = df.set_index('ts_event')[['bid_weight_log', 'ask_weight_log']].resample('5min').mean().plot()
df[df['action'] == 'T'].set_index('ts_event')['price'].resample('5min').mean().plot(ax=ax, secondary_y=True, color='r', label='Price')
ax.set_xlabel('Time')
ax.set_ylabel('Log Weight')
ax.right_ax.set_ylabel('Price')
ax.set_title('Bid and Ask Weight Log Over Time with Trade Prices')
ax.legend(loc='upper left')
ax.right_ax.legend(loc='upper right')
plot.show()

### Calculating trades on the bid and ask side


In [140]:
# add a column with a the traded size by row, with 0 for non trade rows
# this makes the next calculation easier
df["traded_bid_size"] = df["size"] * ((df["action"] == "T") & (df["side"] == "B"))
df["traded_ask_size"] = df["size"] * ((df["action"] == "T") & (df["side"] == "A"))
df["is_trade_bid"] = ((df["action"] == "T") & (df["side"] == "B")) * 1
df["is_trade_ask"] = ((df["action"] == "T") & (df["side"] == "A")) * 1

def add_rolling_trades_cols(period):
    # temporary dataset so we can reindex
    x = df[['ts_event', 'traded_bid_size', 'traded_ask_size', 'is_trade_bid', 'is_trade_ask']].copy()
    x['original_index'] = df.index
    x.set_index("ts_event", inplace=True)

    # calculate
    x[f'rolling_{period}_bid_size'] = x["traded_bid_size"].rolling(period).sum()
    x[f'rolling_{period}_ask_size'] = x["traded_ask_size"].rolling(period).sum()
    x[f'rolling_{period}_bid_cnt'] = x["is_trade_bid"].rolling(period).sum()
    x[f'rolling_{period}_ask_cnt'] = x["is_trade_ask"].rolling(period).sum()

    # merge back to original dataframe
    x.set_index("original_index", inplace=True)
    df[f'rolling_{period}_bid_size'] = x[f'rolling_{period}_bid_size']
    df[f'rolling_{period}_ask_size'] = x[f'rolling_{period}_ask_size']
    df[f'rolling_{period}_bid_cnt'] = x[f'rolling_{period}_bid_cnt']
    df[f'rolling_{period}_ask_cnt'] = x[f'rolling_{period}_ask_cnt']


add_rolling_trades_cols('30s')
add_rolling_trades_cols('5min')

# and cleanup
_ = df.drop(columns=['traded_bid_size', 'traded_ask_size', 'is_trade_bid', 'is_trade_ask'])



In [ ]:
ax = df.set_index('ts_event')[['rolling_30s_bid_size', 'rolling_30s_ask_size']].resample('5min').mean().plot()
df.set_index('ts_event')[['rolling_30s_bid_cnt', 'rolling_30s_ask_cnt']].resample('5min').mean().plot(ax=ax, secondary_y=True)
ax.set_xlabel('Time')
ax.set_ylabel('size')
ax.right_ax.set_ylabel('count')
ax.legend(loc='upper left')
ax.right_ax.legend(loc='upper right')
plot.show()

### Identify the best hedge position for each row

In [ ]:
rdf = df[df['action'] == 'A'].sample(n=5000)
rdf['hedge_buy_exec_idx'] = rdf.apply(lambda row: df.index[(df.index > row.name) & (df['ask_px_00'] <= row['bid_px_00'])].min(), axis=1)
rdf['hedge_sell_exec_idx'] = rdf.apply(lambda row: df.index[(df.index > row.name) & (df['bid_px_00'] >= row['ask_px_00'])].min(), axis=1)

rdf['hedge_buy_stop_idx'] = rdf.apply(lambda row: df.index[(df.index > row.name) & (df['bid_px_00'] >= row['bid_px_00']+0.02)].min(), axis=1)
rdf['hedge_sell_stop_idx'] = rdf.apply(lambda row: df.index[(df.index > row.name) & (df['ask_px_00'] <= row['ask_px_00']-0.02)].min(), axis=1)

rdf.shape

In [ ]:
data = {
    'A': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
    'W': [3, 3, 5, 6, 7, 8, 9, np.nan, np.nan, np.nan],
}
dfo = pd.DataFrame(data)
df2 = dfo[dfo.A % 3 > 0].copy()
print(type(df2.index))
print(df2.index)
shifts = df2['W'].where(~df2['W'].isna(), df2.index).astype(int)

df2['ws'] = shifts-df2.index
print(dfo)
df2.shift(-df2.ws).max(axis=1)


In [166]:
df.index

Index([  80776,   80777,   80778,   80779,   80780,   80781,   80782,   80783,
         80784,   80785,
       ...
       1499990, 1499991, 1499992, 1499993, 1499994, 1499995, 1499996, 1499997,
       1499998, 1499999],
      dtype='int64', length=1419224)

In [173]:
# fix NaN values
rdf['hedge_buy_stop_idx'] = rdf['hedge_buy_stop_idx'].where(~rdf['hedge_buy_stop_idx'].isna(), rdf.index)
rdf['hedge_sell_stop_idx'] = rdf['hedge_sell_stop_idx'].where(~rdf['hedge_sell_stop_idx'].isna(), rdf.index)

# convert idexes to int
rdf['hedge_buy_stop_idx'] = rdf['hedge_buy_stop_idx'].astype(int)
rdf['hedge_sell_stop_idx'] = rdf['hedge_sell_stop_idx'].astype(int)


def optimal_buy_price(row):
    idx = df.index.get_loc(row.name)
    return df.iloc[idx:idx+1+row['hedge_buy_stop_idx']]['ask_px_00'].min()

def optimal_sell_price(row):
    idx = df.index.get_loc(row.name)
    return df.iloc[idx:idx+1+row['hedge_sell_stop_idx']]['bid_px_00'].max()

# calculate ideal position between current point and stop loss point 
rdf['ideal_buy_price'] = rdf.apply(optimal_buy_price, axis=1)
rdf['ideal_sell_price'] = rdf.apply(optimal_sell_price, axis=1)



In [174]:
idx = 0

In [177]:
rdf.iloc[idx:idx+1][['bid_px_00', 'ask_px_00', 'ideal_buy_price', 'ideal_sell_price']]
stopidx = rdf.iloc[idx]['hedge_buy_stop_idx']
row = rdf.iloc[idx]
rowiloc = df.index.get_loc(row.name)
stopiloc = df.index.get_loc(stopidx)
print(row[['bid_px_00', 'ask_px_00', 'ideal_buy_price', 'ideal_sell_price']])
df.iloc[rowiloc:stopiloc][['bid_px_00', 'ask_px_00']]

bid_px_00           143.86
ask_px_00           143.88
ideal_buy_price     142.52
ideal_sell_price    143.97
Name: 1020127, dtype: object


,bid_px_00,ask_px_00
1020127,143.86,143.88
1020128,143.86,143.88
1020129,143.86,143.88
1020130,143.86,143.88
1020131,143.86,143.88
...,...,...
1136927,143.88,143.90
1136928,143.88,143.90
1136929,143.88,143.90
1136930,143.88,143.90


In [ ]:
row = df.loc[1459267]
df.index[(df.index > row.name) & (df['ask_px_00'] <= row['bid_px_00'])].min()
row.ask_px_00, df.loc[row.name:].ask_px_00.min()

### In search of one order triggering more than one execution

In [ ]:
df[(df.action == "T") & (df.side == "B") & 
   (df.shift(1)["ask_px_00"] != df["ask_px_00"])][cols].head()
#   & ((df.shift(-1)["ask_px_00"] == df["ask_px_00"] ) & (df.shift(-1)["ask_sz_00"] == df["ask_sz_00"]-df["size"]))][cols].head()
      

In [ ]:
display_around(882, 10)

In [ ]:
df[(df.action == "T") & (df.side != "N") & (df.shift(-1).action != "C")][cols]

In [ ]:
display_around(206)

### In search of iceberg orders